# Introduction to GeoPandas


## Table of Contents

1. [Points, lines and polygons](#GeoDataFrames)<br>
2. [Spatial relationships](#spatial)<br>
3. [London boroughs](#boroughs)<br>
    3.1. [Load geospatial data](#load1)<br>
    3.2. [Explore data](#explore1)<br>
4. [Open Street Map data (OSM)](#osm)<br>
    4.1. [Load data](#load2)<br>
    4.2. [Explore data](#explore2)<br>


<div class="alert alert-danger" style="font-size:100%">
If you are using <b>Watson Studio</b> to run the workshop you will need to add the project token to your notebook that you created earlier to be able to access the shape files from your Cloud Object Store (COS). 

Click the 3 dots at the top right side of the notebook to insert the project token. This will create a new cell in the notebook that you will need to run first before continuing with the rest of the notebook. If you are sharing this notebook you should remove this cell, else anyone can use you Cloud Object Storage from this project.

If you cannot find the new cell it is probably at the top of this notebook. Scroll up, run the cell and continue with the rest of the notebook below.

</div> 

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt

%matplotlib inline

<a id="GeoDataFrames"></a>
## 1. Points, lines and polygons

A [`GeoDataSeries`](http://geopandas.org/data_structures.html) is a vector where each row is a set of shapes corresponding to one observation. A row may consist of only one shape (like a single polygon) or multiple shapes that are meant to be thought of as one observation.

A `GeoDataFrame` is very similar to a Pandas `DataFrame`, but has an additional column with the shape or `geometry`. You can load a file, or create your own `GeoDataFrame`. 

Below the latitude and longitude of 5 cities are used to create a `POINT` geometry variable that is used to create a `GeoDataFrame` from a `DataFrame`: 

In [ ]:
df = pd.DataFrame({'city':       ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'population': [9787426,  2553379,     2440986,    1777934, 1209143],
        'area':       [1737.9,   630.3,       598.9,      487.8,   368.5 ],
        'latitude':   [51.50853, 53.48095,    52.48142,   53.79648,55.86515],
        'longitude':  [-0.12574, -2.23743,    -1.89983,   -1.54785,-4.25763]})

df['geometry']  = list(zip(df.longitude, df.latitude))

df['geometry'] = df['geometry'].apply(Point)

cities = gpd.GeoDataFrame(df, geometry='geometry')
cities.head()

In [ ]:
list(zip(df.longitude, df.latitude))

Creating a basic map from this data is similar to creating a plot from a Pandas DataFrame by using `.plot()`. Below the column name defines what to use for the colours in the map. (We will come back to creating and editing more maps later) 

In [ ]:
cities.plot(column='population');

As `cities` is still a DataFrame you can apply the same data manipulations, for instance:

In [ ]:
cities['population'].mean()

In [ ]:
cities['area'].min()

In [ ]:
cities['density'] = cities['population']/cities['area']
cities

But there are additional methods you can use (from the [geopandas documentation](http://geopandas.org/data_structures.html#overview-of-attributes-and-methods)):

### Attributes
* `area`: shape area
* `bounds`: tuple of max and min coordinates on each axis for each shape
* `total_bounds`: tuple of max and min coordinates on each axis for entire GeoSeries
* `geom_type`: type of geometry
* `is_valid`: tests if coordinates make a shape that is reasonable geometric shape

### Basic Methods
* `distance(other)`: returns Series with minimum distance from each entry to other
* `centroid`: returns GeoSeries of centroids
* `representative_point()`: returns GeoSeries of points that are guaranteed to be within each geometry. It does NOT return centroids
* `to_crs()`: change coordinate reference system
* `plot()`: plot GeoSeries

### Relationship Tests
* `geom_almost_equals(other)`: is shape almost the same as other (good when floating point precision issues make shapes slightly different)
* `contains(other)`: is shape contained within other
* `intersects(other)`: does shape intersect other


We can explore a few of these with the cities data:

In [ ]:
cities.area

In [ ]:
cities.total_bounds

In [ ]:
cities.geom_type

In [ ]:
cities.distance(cities.geometry[0])

For the other attributes and methods we need some more data. 

* A line between 2 cities by squeezing out the geometry and then creating a LineString
* Circles around the cities by adding a buffer around the points

In [ ]:
london = cities.loc[cities['city'] == 'London', 'geometry'].squeeze()
manchester = cities.loc[cities['city'] == 'Manchester', 'geometry'].squeeze()

line = gpd.GeoSeries(LineString([london, manchester]))
line.plot();

In [ ]:
cities2 = cities.copy()
cities2['geometry'] = cities2.buffer(1)
cities2 = cities2.drop([1, 2])
cities2.head()

In [ ]:
cities2.geometry[0]

In [ ]:
cities2.plot();

And plot all of them together:

In [ ]:
base = cities2.plot(color='lightblue', edgecolor='black')
cities.plot(ax=base, marker='o', color='red', markersize=10);
line.plot(ax=base);

Polygons can be of any shape as you will see later in the workshop, using circles here as a quick example. 

Polygons can contain holes. Let's subtract a small circle from three larger ones to see what that looks like:

In [ ]:
cities3 = cities.copy()
cities3['geometry'] = cities3.buffer(2)
cities3 = cities3.drop([1, 2])

gpd.overlay(cities3, cities2, how='difference').plot();

With these new shapes let's explore some more methods:

In [ ]:
cities2.area

In [ ]:
cities2.bounds

In [ ]:
cities2.centroid

In [ ]:
cities3.representative_point()

<a id="spatial"></a>
## 2. Spatial relationships

What can you do with geospatial relationships?  

There are several functions to check geospatial relationships between geometries: `equals`, `contains`, `crosses`, `disjoint`,`intersects`,`overlaps`,`touches`,`within` and `covers`. These all use the `shapely` package about which you can read more [here](https://shapely.readthedocs.io/en/stable/manual.html#predicates-and-relationships) and some more background on spatial relationships [here](https://en.wikipedia.org/wiki/Spatial_relation).

A few examples:

In [ ]:
cities2.head()

In [ ]:
cities.head()

In [ ]:
cities2.contains(cities.geometry[0])

In [ ]:
cities2.contains(london)

In [ ]:
cities2[cities2.contains(london)]

In [ ]:
cities2[cities2.contains(manchester)]

The inverse of `contains`:

In [ ]:
cities[cities.within(cities2)]

In [ ]:
cities2.intersects(line)

In [ ]:
cities2[cities2.crosses(line)]

In [ ]:
cities2[cities2.disjoint(london)]

<a id="boroughs"></a>
## 3. London boroughs

<a id="load1"></a>
### 3.1 Load geospatial data

Geospatial data comes in many formats, but with GeoPandas you can read most files with just one command. For example this geojson file with the London boroughs: 

In [ ]:
# load data from a url
boroughs = gpd.read_file("https://skgrange.github.io/www/data/london_boroughs.json")
boroughs.head()

<a id="explore1"></a>
### 3.2 Explore  data

In [ ]:
boroughs.plot();

Adding a column will colour the map based on the classes in this column:

In [ ]:
boroughs.plot(column='code');

In [ ]:
boroughs.plot(column='area_hectares');

The boroughs are made up of many districts that you might want to combine. For this example this can be done by adding a new column and then use `.dissolve()`:

In [ ]:
boroughs['all'] = 1
allboroughs = boroughs.dissolve(by='all',aggfunc='sum')
allboroughs.head()

In [ ]:
allboroughs.plot();

To change the size of the map and remove the box around the map, run the below:

In [ ]:
[fig, ax] = plt.subplots(1, figsize=(10, 6))
allboroughs.plot(ax=ax);
ax.axis('off');

Let's combine the data from the Pandas notebook with the boroughs GeoDataFrame:

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/data/london-borough-profiles.csv',encoding = 'unicode_escape')

In [ ]:
df.head()

In [ ]:
boroughs.head()

The columns to join the two tables on are `code` and `Code`. To use the [`join` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html), first the index of both tables has to be set to this column. 

The below adds the columns from `df` to `boroughs`:

In [ ]:
boroughs = boroughs.set_index('code').join(df.set_index('Code'))
boroughs.head()

<div class="alert alert-success">
 <b>EXERCISES</b> <br/> 
  <ol>
  <li>Create a map that shows two regions: Inner and Outer London. </li>
  <li>Create a map of the average gender pay gap for each borough.  </li>
  <li>Create a map or maps with the columns that you are curious about. </li>    
 </ul> 

<b>Tip</b>: you can pick any of the color maps from [here](https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html) to use in your maps.

</div>  




In [ ]:
# answer 1: Inner and Outer London map (add as many cells as you need)


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/answers/geo_answer1a.py

In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/answers/geo_answer1b.py

In [ ]:
# answer 2: average gender pay gap map


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/answers/geo_answer2.py

In [ ]:
# answer 3: your turn to play!


<a id="osm"></a>
## 4. Open Street Map data (OSM)

<a id="load2"></a>
### 4.1 Load OSM data

The Open Street Map data is pre-processed in this [notebook]().

Data is downloaded from http://download.geofabrik.de/europe/great-britain.html and a more detailed decription of the data is [here](http://download.geofabrik.de/osm-data-in-gis-formats-free.pdf).

The data format is a shape file that consists of several files combined into one zip file that can be read directly with GeoPandas:

<div class="alert alert-danger" style="font-size:100%">
If you are using <b>Watson Studio</b> to run the workshop you will get an error with the above, because you have no local files in the data folder. First store the file <b><font face="Courier">london_pois.zip</font></b> in your Cloud Object Store (COS) through the menu at the right of the notebook (if you see no menu, click the <b><font face="Courier">1010</font></b> button at the top first). Then load the data into the notebook by running the following two cells (<b>do not forget to uncomment the second cell</b>):
</div> 

In [ ]:
# define the helper function 
def download_file_to_local(project_filename, local_file_destination=None, project=None):
    """
    Uses project-lib to get a bytearray and then downloads this file to local.
    Requires a valid `project` object.
    
    Args:
        project_filename str: the filename to be passed to get_file
        local_file_destination: the filename for the local file if different
        
    Returns:
        0 if everything worked
    """
    
    project = project
    
    # get the file
    print("Attempting to get file {}".format(project_filename))
    _bytes = project.get_file(project_filename).read()
    
    # check for new file name, download the file
    print("Downloading...")
    if local_file_destination==None: local_file_destination = project_filename
    
    with open(local_file_destination, 'wb') as f: 
        f.write(bytearray(_bytes))
        print("Completed writing to {}".format(local_file_destination))
        
    return 0

In [ ]:
download_file_to_local('london_pois.zip', project=project)
pois = gpd.read_file("zip://./london_pois.zip")

In [ ]:
pois.head()

<a id="explore2"></a>
### 4.2 Explore OSM data

In [ ]:
pois.size

In [ ]:
pois['fclass'].unique()

In [ ]:
pois.plot(column='fclass');

Let's count and plot the number of pubs by borough by:

* checking the coordinate systems of the maps to combine. They need to be the same to use them together.
* extracting the pubs from the `pois` DataFrame
* joining the tables into a temporary table
* counting the number of pubs in each borough
* merging this new table back into the `boroughs` DataFrame

In [ ]:
pois[pois.fclass=='pub'].plot(column='fclass');

The coordinate reference system (CRS) determines how the two-dimensional (planar) coordinates of the geometry objects should be related to actual places on the (non-planar) earth.

In [ ]:
print(pois.crs)
print(boroughs.crs)

In [ ]:
pubs = pois[pois['fclass']=='pub']
pubs.head()

In [ ]:
pubs2 = gpd.sjoin(boroughs,pubs) 
pubs2.head()

In [ ]:
pubs3 = pd.pivot_table(pubs2,index='name_left',columns='fclass',aggfunc={'fclass':'count'})
pubs3.columns = pubs3.columns.droplevel()
pubs3 = pubs3.reset_index()
pubs3.head()

In [ ]:
boroughs = boroughs.merge(pubs3, left_on='name',right_on='name_left')
boroughs = boroughs.drop(columns='name_left')
boroughs.head()

In [ ]:
[fig,ax] = plt.subplots(1, figsize=(12, 8))

boroughs.plot(column='pub',cmap='Blues', edgecolor='black', linewidth=0.5, 
              legend=True, ax=ax, scheme='equal_interval');
ax.axis('off');
ax.set_title('Pubs in London');


A different way to visualize this is with a heatmap:

In [ ]:
import geoplot 
[fig,ax] = plt.subplots(1, figsize=(12, 8))

geoplot.kdeplot(
    pubs, clip=boroughs.geometry, n_levels=10, 
    shade=True, cmap='Greens', ax=ax)
geoplot.polyplot(boroughs, ax=ax, alpha=1, edgecolor='black', linewidth=0.5)

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Explore the data further with GeoPandas. Some suggestions of what to look at: <br/> 
   <ol>
  <li> Create a map that only shows all points of one of the POI classes for one of the boroughs.  </li>
  <li> Add another POI class to the boroughs table.</li> 
  <li> Are the columns in the borough table related to any of the POI classes. Combine the data and try making maps, scatter plots or bar charts to find out.  </li>     
  </ol> 
</div>  

In [ ]:
# answer 1


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/answers/geo_answer3.py

In [ ]:
# answer 2


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/crime-data-workshop/master/answers/geo_answer4.py

In [ ]:
# answer 3: your turn to play! There are no right or wrong answers when exploring new data


Hopefully you got an idea of the possibilities with geospatial data now. There is a lot more to explore with this data. Let me know if you find anything interesting! I am on Twitter as @MargrietGr. 

### Author
Margriet Groenendijk is a Data & AI Developer Advocate for IBM. She develops and presents talks and workshops about data science and AI. She is active in the local developer communities through attending, presenting and organising meetups. She has a background in climate science where she explored large observational datasets of carbon uptake by forests during her PhD, and global scale weather and climate models as a postdoctoral fellow. 

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.